In [1]:
import os

In [ ]:
%pwd

In [3]:
os.chdir('../')

In [ ]:
%pwd

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    unzip_data_dir: Path
    STATUS_FILE:str
    all_schema: str

In [6]:
from src.inproject.constants import *
from src.inproject.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self,
                 config_pathway = CONFIG_FILE_PATH,
                 param_pathway = PARAM_FILE_PATH,
                 schema_pathway = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_pathway)
        self.params = read_yaml(param_pathway)
        self.schema = read_yaml(schema_pathway)

        create_directories([self.config.artifacts_root])

    def get_data_validation_config(self):
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            unzip_data_dir=config.unzip_data_dir,
            STATUS_FILE=config.STATUS_FILE,
            all_schema= schema
        )

        return data_validation_config

In [8]:
import os
import pandas as pd
from inproject.logging import logger

In [9]:
# ~ Component

class DataValidation:
    def __init__(self, config=DataValidationConfig):
        self.config = config

    def validate_all_columns(self)->bool:
        try:
            validation_status=None
            data = pd.read_csv(self.config.unzip_data_dir)
            # print(data)
            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()
            print(all_schema)
            print(all_cols)

            for col in all_cols:
                if col not in all_schema:
                    validation_status=False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f'validation status for columns is {validation_status}')

                else:
                    validation_status=True
                    with open(self.config.STATUS_FILE, 'a') as f:
                        f.write(f'validation status for columns is {validation_status}')

                return validation_status
            
        except Exception as e:
            raise e
        
    def validate_datatypes(self)->bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir)
            all_datatypes = list(data.dtypes)

            all_schema_datatypes = self.config.all_schema.values()

            for datatype in all_datatypes:
                
                if datatype not in all_schema_datatypes:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'a') as f:
                        f.write(f'\nvalidation status for datatypes {validation_status}')
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'a') as f:
                        f.write(f'\nvalidation status for datatype {validation_status}')

                return validation_status
            
        except Exception as e:
            raise e

In [ ]:
#pipeline

try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(data_validation_config)
    data_validation.validate_all_columns()
    data_validation.validate_datatypes()

except Exception as e:
    logger.exception(e)
    raise e